In [1]:
#9Bihar
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from decimal import Decimal
import pandas as pd
import time


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20) 

bus_route_link = []
bus_route_name = []
bus_route_final = []
#bus_route = []
bus_names = [] 
bus_type = [] 
departing_time = []
duration = [] 
Reaching_time = [] 
star_rating = [] 
price = [] 
seat_availablity = []

def jump_pages():
    for page_number in range(1,5):
        find_bus_routes = driver.find_elements(By.CLASS_NAME,'route')
        if page_number < 5:
            for route in find_bus_routes:
                route_name = route.get_attribute("title")
                route_link = route.get_attribute("href")
                bus_route_name.append(route_name)
                bus_route_link.append(route_link)
            try:
                pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DC_117_paginationTable")]')))
                next_page_button=pagination_container.find_element(By.XPATH, f'//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
                action = ActionChains(driver)
                action.move_to_element(next_page_button).perform()
                time.sleep(2)
                next_page_button.click()
            except:
                print('failure')

def scroll(): # function to scroll page from top to bottom
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to the bottom in order to load all the buses
        time.sleep(2)# Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:# If it is the same height then it is at the end of the page
            break
        last_height = new_height

def click_button(): # function to click button for view buses for government buses
    view_buses = driver.find_elements(By.CSS_SELECTOR, '.p-left-10.icon.icon-down')
    if view_buses:
        button_count = len(view_buses)
        for i in range(button_count-1 , -1, -1):
            view_buses[i].click()

def get_bus_details(): # function to get all bus details
    rate = driver.find_elements(By.CSS_SELECTOR,".fare.d-block")#
    seat_available = driver.find_elements(By.CSS_SELECTOR,".column-eight.w-15.fl")
    rating = driver.find_elements(By.CSS_SELECTOR,".column-six.p-right-10.w-10.fl")
    names = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
    b_type = driver.find_elements(By.CSS_SELECTOR,".bus-type.f-12.m-top-16.l-color.evBus")
    start_time = driver.find_elements(By.CSS_SELECTOR,".dp-time.f-19.d-color.f-bold")
    total_time = driver.find_elements(By.CSS_SELECTOR,".dur.l-color.lh-24")
    end_time =  driver.find_elements(By.CSS_SELECTOR,".bp-time.f-19.d-color.disp-Inline")
    
    time.sleep(5)
    print(len(names),len(b_type),len(start_time),len(total_time),len(end_time),len(rating),len(rate),len(seat_available))

    for i in range(0,len(seat_available)):#(len(rating)):
        bus_names.append(names[i].text)
        bus_type.append(b_type[i].text)
        departing_time.append(start_time[i].text)
        duration.append(total_time[i].text)
        Reaching_time.append(end_time[i].text)
        if (rating[i].text) == "New":
            star_rating.append(0.0)
        else:
            star_rating.append((rating[i].text)[0:3])
        price.append((rate[i].text))
        seat_availablity.append((seat_available[i].text)[0:2])
    
    print(len(bus_names),len(bus_type),len(departing_time),len(duration),len(Reaching_time),len(star_rating),len(price),len(seat_availablity))
    print((bus_names),(bus_type),(departing_time),(duration),(Reaching_time),(star_rating),(price),(seat_availablity))


driver.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc")
driver.maximize_window()
time.sleep(5)

jump_pages()

print("Total routes",len(bus_route_name))

for i in range (0,len(bus_route_link)):# used 2 just to check if every thing is working as expected
    driver.get(bus_route_link[i])
    body = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.clearfix.bus-item')))
    click_button()
    scroll()
    time.sleep(5)
    get_bus_details()
    temp_name = []
    temp_link = []
    bus_route = []
    state_service = []
    print(len(bus_names))
    for j in range(len(bus_names)):
        temp_name.append(bus_route_name[i])
        temp_link.append(bus_route_link[i])
        state_service.append("Bihar")
    print(len(temp_name),len(temp_link))
    bus_route = list(zip(state_service,temp_name,temp_link,bus_names,bus_type,departing_time,duration,Reaching_time,star_rating,price,seat_availablity))
    bus_route_final.extend(bus_route)
    print(i,bus_route_name[i],"completed")
    temp_name.clear()
    temp_link.clear()
    state_service.clear()
    bus_route.clear()
    bus_names.clear()
    bus_type.clear()
    departing_time.clear()
    duration.clear()
    Reaching_time.clear()
    star_rating.clear()
    price.clear()
    seat_availablity.clear()
    
df_bus_details_1 = pd.DataFrame(bus_route_final, columns=["Service Name","Route_name", "Route_link", "Bus_names","Bus_type","Departing_time","Duration","Reaching_time","Star_rating","Price","Seat_availablity"])
df_bus_details_1['Seat_availablity']=df_bus_details_1['Seat_availablity'].astype(int)
df_bus_details_1['Price'] = df_bus_details_1['Price'].str.replace('INR ', '', regex=False)
df_bus_details_1['Star_rating'] = df_bus_details_1['Star_rating'].str.replace(' ', '', regex=False)
df_bus_details_1['Price'] = df_bus_details_1['Price'].astype(float)
df_bus_details_1['Star_rating'] = df_bus_details_1['Star_rating'].fillna(0.0)

failure
Total routes 38
20 20 20 20 20 20 20 20
20 20 20 20 20 20 20 20
['Bihar state road transport corporation (BSRTC) - 146158', 'Jai Mata Di', 'Jai Mata Di', 'Jai Mata Di', 'Jai Mata Di', 'Jai Hanuman Travels', 'Jai Mata Di Tour and Travels', 'Saurabh Travels', 'Saurabh Travels', 'Jai Hanuman Travels', 'Saurabh Travels', 'Saurabh Travels', 'Saurabh Travels', 'Saurabh Travels', 'Saurabh Travels', 'Saurabh Travels', 'Vivek Travels', 'Vivek Travels', 'Jai Mata Di Tour and Travels', 'V.I.P Travels'] ['Non AC Seater (2+2)', 'AC Seater (2+2)', 'AC Seater (2+2)', 'AC Seater (2+2)', 'AC Seater (2+2)', 'A/C Seater / Sleeper (2+2)', 'A/C Seater / Sleeper (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater / Sleeper (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater Push Back (2+2)', 'A/C Seater (2+2)', 'A/C Seater / Sleeper (2+2)', 'A/C Seater / S

In [9]:
pd.set_option('display.max_rows',None)
df_bus_details_1

,Service Name,Route_name,Route_link,Bus_names,Bus_type,Departing_time,Duration,Reaching_time,Star_rating,Price,Seat_availablity
0,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),09:00,06h 00m,15:00,3.3,285.00,33
1,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di,AC Seater (2+2),05:40,06h 25m,12:05,3.7,405.00,40
2,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di,AC Seater (2+2),05:43,06h 17m,12:00,3.7,405.00,41
3,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di,AC Seater (2+2),05:45,06h 05m,11:50,3.7,405.00,41
4,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di,AC Seater (2+2),05:47,06h 16m,12:03,3.7,405.00,41
5,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.6,495.00,54
6,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,05h 20m,04:00,3.4,450.00,28
7,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Saurabh Travels,A/C Seater Push Back (2+2),07:00,05h 30m,12:30,1.0,450.00,42
8,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Saurabh Travels,A/C Seater Push Back (2+2),07:12,05h 18m,12:30,1.0,450.00,42
9,Bihar,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),09:30,06h 30m,16:00,2.4,450.00,59


In [11]:
#df_bus_details_1['Star_rating'] = df_bus_details_1['Star_rating'].astype(float)
df_bus_details_1.to_csv("Bihar.csv",index = False)